# Banque : ANN

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3: 13].values
y = dataset.iloc[:, 13].values

In [3]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
ct = ColumnTransformer([("categorical_features", OneHotEncoder(),[1])], remainder="passthrough")
X = ct.fit_transform(X)
X = X[:,1:]

In [4]:

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Importation des modules de keras

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout


Using TensorFlow backend.


### Initialisation

In [6]:
classifier = Sequential()

# Ajouter la couche d'entrée et une couche cachée

In [7]:
classifier.add(Dense(units=6, activation='relu', kernel_initializer='uniform', input_dim=11))


In [ ]:
classifier.add(Dropout(rate=0.1))

# Ajouter une deuxième couche cachée

In [8]:
classifier.add(Dense(units=6, activation='relu', kernel_initializer='uniform'))

In [ ]:
classifier.add(Dropout(rate=0.1))

# Ajouter une couche de sortie

In [9]:
classifier.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))

In [10]:
classifier.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

## Entrainer le réseau de neuronne

In [11]:
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
8000/8000 [==============================] - 2s 302us/step - loss: 0.4853 - accuracy: 0.7956
Epoch 2/100
8000/8000 [==============================] - 2s 263us/step - loss: 0.4273 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 257us/step - loss: 0.4215 - accuracy: 0.8011
Epoch 4/100
8000/8000 [==============================] - 2s 252us/step - loss: 0.4177 - accuracy: 0.8248
Epoch 5/100
8000/8000 [==============================] - 2s 240us/step - loss: 0.4152 - accuracy: 0.8276
Epoch 6/100
8000/8000 [==============================] - 2s 239us/step - loss: 0.4136 - accuracy: 0.8324
Epoch 7/100
8000/8000 [==============================] - 2s 257us/step - loss: 0.4123 - accuracy: 0.83250s - loss: 0.4117 - accuracy: 0.83
Epoch 8/100
8000/8000 [==============================] - 2s 237us/step - loss: 0.4110 - accuracy: 0.8329
Epoch 9/100
8000/8000 [==============================] - 2s 288us/step - loss: 0.4099 - accuracy: 0.8341
Epoch 10/100
8000/800

## Prediction

In [12]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [13]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
cm

array([[1548,   47],
       [ 269,  136]], dtype=int64)

Faire une prédiction : <br>
Pays : France <br>
Score de crédit : 600 <br>
Genre : Masculin <br>
Âge : 40 ans <br>
Durée depuis entrée dans la banque : 3 ans <br>
Balance : 60000 € <br>
Nombre de produits : 2 <br>
Carte de crédit ? Oui <br>
Membre actif ? : Oui <br>
Salaire estimé : 50000 € <br>

In [14]:

new_prediction = classifier.predict(sc.transform(np.array([[0, 0, 600, 0, 40, 3, 60000, 2, 1 , 1 , 50000]])))

In [15]:
new_prediction = (new_prediction > 0.5)
new_prediction

array([[False]])

In [16]:
cm = confusion_matrix(y_test,y_pred)
cm

array([[1548,   47],
       [ 269,  136]], dtype=int64)

In [17]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [18]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units=6, activation='relu', kernel_initializer='uniform', input_dim=11))
    classifier.add(Dropout(rate=0.1))
    classifier.add(Dense(units=6, activation='relu', kernel_initializer='uniform'))
    classifier.add(Dropout(rate=0.1))
    classifier.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))
    classifier.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return classifier

In [19]:
classifier = KerasClassifier(build_fn=build_classifier, batch_size=10, epochs=100)


In [20]:
precisions = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10, n_jobs=-1)

In [21]:
precisions.mean()

0.842125004529953

ERROR! Session/line number was not unique in database. History logging moved to new session 344


In [22]:
precisions.std()

0.019056244935491733

In [23]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units=6, activation='relu', kernel_initializer='uniform', input_dim=11))
    # classifier.add(Dropout(rate=0.1))
    classifier.add(Dense(units=6, activation='relu', kernel_initializer='uniform'))
    # classifier.add(Dropout(rate=0.1))
    classifier.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))
    classifier.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
    return classifier

classifier = KerasClassifier(build_fn=build_classifier)

parameters = {
    "batch_size":[25,32], 
    "epochs": [100,500],
    "optimizer": ['rmsprop','adam']
}

grid_search = GridSearchCV(estimator=classifier,param_grid=parameters, scoring='accuracy',cv=10)

grid_search = grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)

Epoch 1/100
7200/7200 [==============================] - 0s 47us/step - loss: 0.6040 - accuracy: 0.7953
Epoch 2/100
7200/7200 [==============================] - 0s 34us/step - loss: 0.4545 - accuracy: 0.7971
Epoch 3/100
7200/7200 [==============================] - 0s 33us/step - loss: 0.4365 - accuracy: 0.7971
Epoch 4/100
7200/7200 [==============================] - 0s 43us/step - loss: 0.4305 - accuracy: 0.7971 0s - loss: 0.4312 - accuracy
Epoch 5/100
7200/7200 [==============================] - 0s 39us/step - loss: 0.4272 - accuracy: 0.7971
Epoch 6/100
7200/7200 [==============================] - 0s 32us/step - loss: 0.4240 - accuracy: 0.7971
Epoch 7/100
7200/7200 [==============================] - 0s 38us/step - loss: 0.4209 - accuracy: 0.8004
Epoch 8/100
7200/7200 [==============================] - 0s 38us/step - loss: 0.4178 - accuracy: 0.8222
Epoch 9/100
7200/7200 [==============================] - 0s 39us/step - loss: 0.4151 - accuracy: 0.8265
Epoch 10/100
7200/7200 [===========